In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-tfrecords/val.tfrecords
/kaggle/input/dataset-tfrecords/train.tfrecords
/kaggle/input/dataset-tfrecords/test.tfrecords


In [2]:
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import glob
import cv2
import sys
from random import shuffle

In [3]:
 class DataSequenceReader():
    def __init__(self, batch_size, num_epochs):
        self.batch_size = batch_size
        self.num_epochs = num_epochs
    
    def parse_sequence(self, sequence_example):

        sequence_features = {'Images': tf.io.FixedLenSequenceFeature([], dtype=tf.string),
                          'Labels': tf.io.FixedLenSequenceFeature([], dtype=tf.int64)}

        context_features = {'length': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'height': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'width': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'depth': tf.io.FixedLenFeature([], dtype=tf.int64)}
        
        context, sequence = tf.io.parse_single_sequence_example(
            sequence_example, context_features=context_features, sequence_features=sequence_features)
        # get features context
        seq_length = tf.cast(context['length'], dtype = tf.int32)
        im_height = tf.cast(context['height'], dtype = tf.int32)
        im_width = tf.cast(context['width'], dtype = tf.int32)
        im_depth = tf.cast(context['depth'], dtype = tf.int32)

        # encode image
        image = tf.map_fn(lambda x: tf.io.decode_jpeg(x, 1), sequence['Images'], tf.uint8)
        image = tf.cast(image, dtype = tf.float32)
        image = tf.reshape(image, shape=(seq_length, im_height, im_width, im_depth))
        image = tf.image.per_image_standardization(image)
        label = tf.cast(sequence['Labels'], dtype = tf.int32)

        return image, label[0]
    
    def read_batch(self, filename):
        

        dataset = tf.data.TFRecordDataset(filename)
        dataset = dataset.map(self.parse_sequence)

        dataset = dataset.shuffle(buffer_size=10000)

        dataset = dataset.batch(self.batch_size, drop_remainder=True)
        
#         dataset = dataset.repeat(self.num_epochs)

        return dataset

In [4]:
# My model
class Conv3DModel(tf.keras.Model):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        # Convolutions
        self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')

        # LSTM & Flatten
        self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
        self.flatten =  tf.keras.layers.Flatten(name="flatten")

        # Dense layers
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
        self.out = tf.keras.layers.Dense(30, activation='softmax', name="output")


    def call(self, x):
#         print(x)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.convLSTM(x)
        #x = self.pool2(x)
        #x = self.conv3(x)
        #x = self.pool3(x)
        x = self.flatten(x)

        x = self.d1(x)
        x= self.out(x)
#         print(x.shape)
        return x

In [5]:
with tf.device('/device:GPU:0'):
#with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = Conv3DModel()
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [7]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(image)
        # Get the error/loss on these predictions
        loss = loss_fn(targets, predictions)
    # Compute the gradient which respect to the loss
    grads = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

In [8]:
@tf.function
def valid_step(image, targets):
    predictions = model(image)
    t_loss = loss_fn(targets, predictions)
    # Set the metrics for the test
    valid_loss(t_loss)
    valid_accuracy(targets, predictions)

In [9]:
batch_size =32
num_epochs = 50
data = DataSequenceReader(batch_size, num_epochs)

train_batch = data.read_batch('/kaggle/input/dataset-tfrecords/train.tfrecords')
val_batch = data.read_batch('/kaggle/input/dataset-tfrecords/val.tfrecords')

count=0

training_acc =[]
validation_acc = []

for i in range(50):
    for images, labels in train_batch:
        train_step(images, labels)

        
    for images, labels in val_batch:
        valid_step(images, labels)
    
    print("--Train step done--")
    count=count+1
    print(count)
    
    print(train_loss.result(), train_accuracy.result()*100)
    print(valid_loss.result(), valid_accuracy.result()*100)

    training_acc.append(float(train_accuracy.result()*100))
    validation_acc.append(float(valid_accuracy.result()*100))
    

    train_loss.reset_states()
    train_accuracy.reset_states()

    valid_loss.reset_states()
    valid_accuracy.reset_states()

--Train step done--
1
tf.Tensor(2.1218958, shape=(), dtype=float32) tf.Tensor(35.94172, shape=(), dtype=float32)
tf.Tensor(1.4798125, shape=(), dtype=float32) tf.Tensor(52.76182, shape=(), dtype=float32)
--Train step done--
2
tf.Tensor(1.2373847, shape=(), dtype=float32) tf.Tensor(59.51845, shape=(), dtype=float32)
tf.Tensor(1.1300117, shape=(), dtype=float32) tf.Tensor(64.189186, shape=(), dtype=float32)
--Train step done--
3
tf.Tensor(0.90494436, shape=(), dtype=float32) tf.Tensor(69.84979, shape=(), dtype=float32)
tf.Tensor(1.0451857, shape=(), dtype=float32) tf.Tensor(66.79899, shape=(), dtype=float32)
--Train step done--
4
tf.Tensor(0.6727922, shape=(), dtype=float32) tf.Tensor(76.95207, shape=(), dtype=float32)
tf.Tensor(1.0378667, shape=(), dtype=float32) tf.Tensor(67.58446, shape=(), dtype=float32)
--Train step done--
5
tf.Tensor(0.4729258, shape=(), dtype=float32) tf.Tensor(83.840576, shape=(), dtype=float32)
tf.Tensor(1.1224511, shape=(), dtype=float32) tf.Tensor(66.75675, sh

In [10]:
model.summary()

Model: "conv3d_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               multiple                  896       
_________________________________________________________________
max_pooling3d (MaxPooling3D) multiple                  0         
_________________________________________________________________
conv1 (Conv3D)               multiple                  55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 multiple                  0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    multiple                  149920    
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
d1 (Dense)                   multiple                 